In [39]:
import numpy as np
import torch
import tntorch as tn

#PARAMETRI GAUSSIANA

num_dimensions = 1
mu = 0.10   # Mean vector
cov_matrix = 0.20  # Covariance matrix 

#PARAMETRI DISCRETIZZAZIOINE

d = 10                 # qubit
m = 2**d               # dimensioni discretizzazione

#GAUSSIANA DISCRETA

domain_np = np.linspace(mu - 3*np.sqrt(cov_matrix), mu + 3*np.sqrt(cov_matrix), m)
def gaussian(x):
    return 1/(np.sqrt(2*np.pi*cov_matrix))*np.exp(-0.5*((x-mu)**2)/np.sqrt(cov_matrix))



vec =  np.array([gaussian(x) for x in domain_np])    # vettore probabilità discreta

shape = (2,)*d         # (2,2,2,2)
A = vec.reshape(shape) #tensore numpy
T=tn.Tensor(A)      #tensore torch


#CREAZIONE TENSOR TRAIN

TTrain = tn.cross(
    function=lambda x: x,   # identità su ciascuna fibra
    tensors=[T],            # lista di un solo tensore               # tolleranza desiderata
    rmax=8,                 # rank massimo ammesso
)



print(TTrain)


cores_torch = TTrain.cores
cores = [c.cpu().numpy() for c in cores_torch]

print(len(cores))
print(cores[0])
print(cores[2].shape)

cross device is cpu
Functions that require cross-approximation can be accelerated with the optional maxvolpy package, which can be installed by 'pip install maxvolpy'. More info is available at https://bitbucket.org/muxas/maxvolpy.
Cross-approximation over a 10D domain containing 1024 grid points:
iter: 0  | eps: 6.524e-01 | time:   0.0111 | largest rank:   1
iter: 1  | eps: 2.339e-05 | time:   0.0282 | largest rank:   4
iter: 2  | eps: 9.670e-11 | time:   0.0476 | largest rank:   7 <- converged: eps < 1e-06
Did 1582 function evaluations, which took 4.268e-05s (3.707e+07 evals/s)

10D TT tensor:

  2   2   2   2   2   2   2   2   2   2
  |   |   |   |   |   |   |   |   |   |
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9)
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
1   2   4   7   7   7   7   7   4   2   1

10
[[[0.89206034 0.1192324 ]
  [0.1192324  0.89206034]]]
(4, 2, 7)


In [40]:
W = []
nk = []
for k in range (len(cores)):
     nk.append(len(cores[k-1][0,0,:]))
print(nk)

for k in range(len(cores)):
    # reshape & SVD 
    j, i, n_k = cores[k].shape
    cores[k] = np.reshape(cores[k], (j * i, n_k))
    U, S, V = np.linalg.svd(cores[k])
    R = np.diag(S) @ V
    if k != len(cores) - 1:
        cores[k+1] = np.einsum('j r,r i k->j i k', R, cores[k+1])

    # Now, we are going to compute on which qubits U acts
    start = k + 1
    mn    = min(start, int(np.log2(nk[k])))
    diff  = int(start - mn)
    # qubit indices from diff up to start (inclusive)
    qubits = list(range(diff-1, start))

    # append both U and the qubit list
    W.append([U, qubits])

for k, (U, qubits) in enumerate(W):
    print(f"Unitary {k}: shape={U.shape}, acts on qubits {qubits}")

print(W[0][1])


[1, 2, 4, 7, 7, 7, 7, 7, 4, 2]
Unitary 0: shape=(2, 2), acts on qubits [0]
Unitary 1: shape=(4, 4), acts on qubits [0, 1]
Unitary 2: shape=(8, 8), acts on qubits [0, 1, 2]
Unitary 3: shape=(14, 14), acts on qubits [1, 2, 3]
Unitary 4: shape=(14, 14), acts on qubits [2, 3, 4]
Unitary 5: shape=(14, 14), acts on qubits [3, 4, 5]
Unitary 6: shape=(14, 14), acts on qubits [4, 5, 6]
Unitary 7: shape=(14, 14), acts on qubits [5, 6, 7]
Unitary 8: shape=(8, 8), acts on qubits [6, 7, 8]
Unitary 9: shape=(4, 4), acts on qubits [8, 9]
[0]
